In [16]:
import numpy
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

M = 5
N = 15
max_iter = 300
epsilon = 0.001


def f(x, y):
    # return 0
    return 2 * np.sin(x) * np.cos(y)


def p(x, y):
    return 1


def q(x, y):
    return 1


def mu_0(x, y):
    # return np.exp(x) * np.sin(y)
    return np.sin(x) * np.cos(y)


def u_exact(x, y):
    # return np.exp(x) * np.sin(y)
    return np.sin(x) * np.cos(y)


def u_exact_matrix(n, m):
    result = np.zeros((n + 1, m + 1))
    for i in range(n + 1):
        for j in range(m + 1):
            result[i][j] = u_exact(x(i, n), y(j, m))
    return result


# x_left = 0
# x_right = 1
# y_left = 0
# y_right = np.pi


x_left = 0
x_right = np.pi
y_left = 0
y_right = 1


In [17]:


def h_x(n):
    return (x_right - x_left) / n


def h_y(m):
    return (y_right - y_left) / m


def a_eigen_values(n, m, c1=1, c2=1, d1=1, d2=1):
    a_lowest_ev = 4 * c1 * np.square(np.sin(np.pi / (2 * n))) / h_x2(n) + 4 * d1 * np.square(
        np.sin(np.pi / (2 * m))) / h_y2(m)

    a_higest_ev = 4 * c2 * np.square(np.cos(np.pi / (2 * n))) / h_x2(n) + 4 * d2 * np.square(
        np.cos(np.pi / (2 * m))) / h_y2(m)
    return a_lowest_ev, a_higest_ev


def tau(n, m):
    d, D = a_eigen_values(n, m)
    return 2 / (d + D)


def h_x2(n):
    return h_x(n) * h_x(n)


def h_y2(m):
    return h_y(m) * h_y(m)


def x(i, n):
    return x_left + i * h_x(n)


def y(j, m):
    return y_left + j * h_y(m)


def Lh(n, m, u):
    result = np.zeros((n + 1, m + 1))
    for i in range(1, n):
        for j in range(1, m):
            result[i][j] = p_i_plus_half(i, j, n, m) * (u(i + 1, j) - u(i, j)) / h_x2(n) - \
                           p_i_minus_half(i, j, n, m) * (u(i, j) - u(i - 1, j)) / h_x2(n) + \
                           q_j_plus_half(i, j, n, m) * (u(i, j + 1) - u(i, j)) / h_y2(m) - \
                           q_j_minus_half(i, j, n, m) * (u(i, j) - u(i, j - 1)) / h_y2(m)
    return - result


def f_matrix(n, m):
    result = np.zeros((n + 1, m + 1))
    for i in range(n + 1):
        for j in range(m + 1):
            result[i][j] = f(x(i, n), y(j, m))
    return result


def nevyazka(n, m, u):
    return norm(Lh(n, m, u) - f_matrix(n, m))


def norm(matrix):
    return numpy.max(np.abs(matrix[1:- 1, 1:- 1]))


def p_i_plus_half(i, j, n, m):
    return p(x(i, n) + h_x(n) / 2, y(j, m))


def p_i_minus_half(i, j, n, m):
    return p(x(i, n) - h_x(n) / 2, y(j, m))


def q_j_plus_half(i, j, n, m):
    return q(x(i, n), y(j, m) + h_y(m) / 2)


def q_j_minus_half(i, j, n, m):
    return q(x(i, n), y(j, m) - h_y(m) / 2)


def get_u_0(n, m):
    u_0 = np.zeros((n + 1, m + 1))
    for j in range(m + 1):
        u_0[0, j] = mu_0(x(0, n), y(j, m))
        u_0[n, j] = mu_0(x(n - 1, n), y(j, m))
    for i in range(n + 1):
        u_0[i, 0] = mu_0(x(i, n), y(0, m))
        u_0[i, m] = mu_0(x(i, n), y(m - 1, m))
    return u_0


def map_matrix(n, m, u_current, u_prev, iteration):
    for i in range(1, n):
        for j in range(1, m):
            u_current[i, j] = iteration(i, j, u_current, u_prev, n, m)


def iterate_using_element_map(n, m, iterations, iteration, epsilon, rho):
    return iterate_using_matrix_map(n, m, iterations,
                                    lambda nn, mm, u_current, u_prev: map_matrix(nn, mm, u_current, u_prev, iteration)
                                    , epsilon, rho)


def iterate_using_matrix_map(n, m, iterations, map_matrix, epsilon, rho):
    u_star = u_exact_matrix(n, m)
    nevyazka_exact = nevyazka(n, m, lambda i, j: u_star[i, j])
    print("1)Мера аппроксимации дифференциального уравнения разностной схемой на точном решении")
    print(nevyazka_exact)

    u0 = get_u_0(n, m)

    nevyazka_zero = nevyazka(n, m, lambda i, j: u0[i, j])
    error_zero = norm(u0 - u_star)
    print("2)Норма невязки нулевого приближения")
    print(nevyazka_zero)

    table = []

    u_prev = u0
    diff_prev = 0
    u_current = numpy.copy(u0)
    for k in range(iterations):
        map_matrix(n, m, u_current, u_prev)

        diff_norm = norm(u_prev - u_current)
        abs_nevyazka = nevyazka(n, m, lambda i, j: u_current[i, j])
        rel_nevyazka = abs_nevyazka / nevyazka_zero
        abs_error = norm(u_star - u_current)
        rel_error = abs_error / error_zero
        apost_est = rho * diff_norm / (1 - rho)
        table_record = [k, abs_nevyazka, rel_nevyazka, abs_error, rel_error, diff_norm, apost_est,
                        diff_norm / diff_prev if diff_prev != 0 else np.NaN]
        table.append(table_record)

        report = pd.DataFrame(table)
        report.columns = ["iter", "discrepancy", "rel. d.", "error", "rel.error", "diff", "apost.est.", "rho_k"]

        u_prev = np.copy(u_current)
        diff_prev = diff_norm

        solution = pd.DataFrame(u_prev)
        solution.columns = [y(j, m) for j in range(m + 1)]
        solution.index = [x(i, n) for i in range(n + 1)]

        if (apost_est < epsilon):
            break

    return (solution, report)



In [18]:

def simple_iteration(n, m, iterations, epsilon, rho):
    return iterate_using_element_map(n, m, iterations, simple_iteration_iteration, epsilon, rho)


def simple_iteration_iteration(i, j, u_current, u_prev, n, m):
    return (p_i_minus_half(i, j, n, m) * u_prev[i - 1, j] / h_x2(n) +
            p_i_plus_half(i, j, n, m) * u_prev[i + 1, j] / h_x2(n) +
            q_j_minus_half(i, j, n, m) * u_prev[i, j - 1] / h_y2(m) +
            q_j_plus_half(i, j, n, m) * u_prev[i, j + 1] / h_y2(m) +
            f(x(i, n), y(j, m))) / (
                   p_i_minus_half(i, j, n, m) / h_x2(n) +
                   p_i_plus_half(i, j, n, m) / h_x2(n) +
                   q_j_minus_half(i, j, n, m) / h_y2(m) +
                   q_j_plus_half(i, j, n, m) / h_y2(m)
           )


d, D = a_eigen_values(N, M)
xi = np.divide(d, D)
rho_simple = (1 - xi) / (1 + xi)
print()
print("Метод простой итерации ")
u_si, table = simple_iteration(N, M, max_iter, epsilon, rho_simple)
print("3)Оценка количества итераций")

print(np.log(1 / epsilon) / (2 * xi))
print("4)спектральный радиус")
print(rho_simple)
print("5)")
print(table)
print("Приблеженное решение")
print(u_si.loc[::int(N / 5), 0::int(M / 5)])



Метод простой итерации 
1)Мера аппроксимации дифференциального уравнения разностной схемой на точном решении
0.006802376520983255
2)Норма невязки нулевого приближения
26.812442725370367
3)Оценка количества итераций
59.164644398508145
4)спектральный радиус
0.8896851114628631
5)
    iter  discrepancy   rel. d.     error  rel.error      diff  apost.est.  \
0      0    12.988054  0.484404  0.896851   0.920132  0.280481    2.262067   
1      1     8.872145  0.330897  0.810067   0.831096  0.135866    1.095754   
2      2     8.233741  0.307087  0.717257   0.735876  0.092810    0.748510   
3      3     7.367149  0.274766  0.631125   0.647508  0.086132    0.694651   
4      4     6.528604  0.243492  0.554058   0.568441  0.077067    0.621539   
5      5     5.777125  0.215464  0.485764   0.498374  0.068295    0.550794   
6      6     5.117007  0.190844  0.425338   0.436380  0.060434    0.487395   
7      7     4.537353  0.169226  0.371838   0.381491  0.053528    0.431703   
8      8     4.0288

In [19]:


def seidel(n, m, iterations, epsilon, rho):
    return iterate_using_element_map(n, m, iterations, seidel_iteration, epsilon, rho)


def seidel_iteration(i, j, u_current, u_prev, n, m):
    return (p_i_minus_half(i, j, n, m) * u_current[i - 1, j] / h_x2(n) +
            p_i_plus_half(i, j, n, m) * u_prev[i + 1, j] / h_x2(n) +
            q_j_minus_half(i, j, n, m) * u_current[i, j - 1] / h_y2(m) +
            q_j_plus_half(i, j, n, m) * u_prev[i, j + 1] / h_y2(m) +
            f(x(i, n), y(j, m))) / (
                   p_i_minus_half(i, j, n, m) / h_x2(n) +
                   p_i_plus_half(i, j, n, m) / h_x2(n) +
                   q_j_minus_half(i, j, n, m) / h_y2(m) +
                   q_j_plus_half(i, j, n, m) / h_y2(m)
           )


print()
print("Метод Зейделя")
rho_seil = np.square((1 - xi) / (1 + xi))
u_seidel, table3 = seidel(N, M, max_iter, epsilon, rho_seil)
print("3)Оценка количества итераций")

print(np.log(1 / epsilon) / (4 * xi))
print("4)спектральный радиус")
print(rho_seil)
print("5)")
print(table3)
print("Приблеженное решение")
print(u_seidel.loc[0:N + 1:int(N / 5), 0:M + 1:int(M / 5)])



Метод Зейделя
1)Мера аппроксимации дифференциального уравнения разностной схемой на точном решении
0.006802376520983255
2)Норма невязки нулевого приближения
26.812442725370367
3)Оценка количества итераций
29.582322199254072
4)спектральный радиус
0.7915395975586872
5)
    iter  discrepancy   rel. d.     error  rel.error      diff  apost.est.  \
0      0    12.017803  0.448217  0.769754   0.789736  0.366766    1.392638   
1      1     7.173469  0.267543  0.644654   0.661389  0.162879    0.618462   
2      2     6.313194  0.235458  0.521249   0.534781  0.140259    0.532572   
3      3     5.276960  0.196810  0.412160   0.422859  0.111268    0.422492   
4      4     4.276649  0.159502  0.320856   0.329185  0.091598    0.347805   
5      5     3.432394  0.128015  0.246201   0.252593  0.074655    0.283469   
6      6     2.745845  0.102409  0.185810   0.190633  0.060392    0.229312   
7      7     2.194365  0.081841  0.138037   0.141621  0.048593    0.184510   
8      8     1.753392  0.0653

In [20]:


def upper_relaxation(n, m, iterations, epsilon, rho):
    return iterate_using_element_map(n, m, iterations, upper_relaxation_iteration, epsilon, rho)


def upper_relaxation_iteration(i, j, u_current, u_prev, n, m):
    d, D = a_eigen_values(N, M)
    xi = np.divide(d, D)
    ro = (1 - xi) / (1 + xi)
    omega = 2 / (1 + np.sqrt(1 - ro * ro))
    return u_prev[i, j] + omega * (
            f(x(i, n), y(j, m)) +
            p_i_plus_half(i, j, n, m) * (u_prev[i + 1, j] - u_prev[i, j]) / h_x2(n) -
            p_i_minus_half(i, j, n, m) * (u_prev[i, j] - u_current[i - 1, j]) / h_x2(n) +
            q_j_plus_half(i, j, n, m) * (u_prev[i, j + 1] - u_prev[i, j]) / h_y2(m) -
            q_j_minus_half(i, j, n, m) * (u_prev[i, j] - u_current[i, j - 1]) / h_y2(m)
    ) / (
                   p_i_minus_half(i, j, n, m) / h_x2(n) +
                   p_i_plus_half(i, j, n, m) / h_x2(n) +
                   q_j_minus_half(i, j, n, m) / h_y2(m) +
                   q_j_plus_half(i, j, n, m) / h_y2(m)
           )


print()
print("Метод верхней релаксации")
d, D = a_eigen_values(N, M)
xi = np.divide(d, D)
ro = (1 - xi) / (1 + xi)
omega = 2 / (1 + np.sqrt(1 - ro * ro))
u_upper, table4 = upper_relaxation(N, M, max_iter, epsilon, omega - 1)
print("3)Оценка количества итераций")
print(np.log(1 / epsilon) / (np.sqrt(xi)))
print("4)спектральный радиус")
print(omega - 1)
print("5)")
print(table4)
print("Приблеженное решение")
print(u_upper.loc[0:N + 1:int(N / 5), 0:M + 1:int(M / 5)])



Метод верхней релаксации
1)Мера аппроксимации дифференциального уравнения разностной схемой на точном решении
0.006802376520983255
2)Норма невязки нулевого приближения
26.812442725370367
3)Оценка количества итераций
28.590029194559964
4)спектральный радиус
0.3730844573627208
5)
    iter  discrepancy   rel. d.     error  rel.error      diff  apost.est.  \
0      0    11.961787  0.446128  0.623209   0.639387  0.566890    0.337363   
1      1     7.731454  0.288353  0.432848   0.444084  0.336258    0.200111   
2      2     4.701748  0.175357  0.244409   0.250753  0.201360    0.119831   
3      3     2.951772  0.110090  0.126810   0.130102  0.126145    0.075070   
4      4     1.706240  0.063636  0.118665   0.121746  0.075175    0.044738   
5      5     1.001163  0.037339  0.119739   0.122847  0.044189    0.026297   
6      6     0.584294  0.021792  0.120064   0.123181  0.025941    0.015437   
7      7     0.341111  0.012722  0.120161   0.123280  0.015063    0.008964   
8      8     0.194

In [21]:


def triangle(n, m, iterations, epsilon, rho):
    return iterate_using_matrix_map(n, m, iterations, triangle_iteration, epsilon, rho)


def triangle_iteration(n, m, u_current, u_prev):
    W = np.zeros((n + 1, m + 1))
    Wk = np.zeros((n + 1, m + 1))
    R = -Lh(n, m, lambda i, j: u_prev[i, j]) + f_matrix(n, m)
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            W[i, j] = (k1 * W[i - 1, j] + k2 * W[i, j - 1] + R[i, j]) / (1 + k2 + k1)
    for i in reversed(range(1, n)):
        for j in reversed(range(1, m)):
            Wk[i, j] = (k1 * Wk[i + 1, j] + k2 * Wk[i, j + 1] + W[i, j]) / (1 + k2 + k1)

    for i in reversed(range(1, n)):
        for j in reversed(range(1, m)):
            Wk[i, j] = (k1 * Wk[i + 1, j] + k2 * Wk[i, j + 1] + W[i, j]) / (1 + k2 + k1)

    for i in range(1, n):
        for j in range(1, m):
            u_current[i, j] = u_prev[i, j] + Wk[i, j] * tau


print()
print("Метод попеременно-треугольной итерации")
d, D = a_eigen_values(N, M)
eta = np.divide(d, D)
omga = 2 / np.sqrt(d * D)
gma1 = d / (2 + 2 * np.sqrt(eta))
gma2 = d / (4 * np.sqrt(eta))
tau = 2 / (gma1 + gma2)
k1 = omga / h_x2(N)
k2 = omga / h_y2(M)
xsi = gma1 / gma2
ro = (1 - xsi) / (1 + xsi)

u_triangle, table5 = triangle(N, M, max_iter, epsilon, ro)

print("3)Оценка количества итераций")
print(np.log(1 / epsilon) / (np.log(1 / ro)))
print("4)спектральный радиус")
print(ro)
print("5)")
print(table5)
print("Приблеженное решение")
print(u_triangle.loc[0:N + 1:int(N / 5), 0:M + 1:int(M / 5)])



Метод попеременно-треугольной итерации
1)Мера аппроксимации дифференциального уравнения разностной схемой на точном решении
0.006802376520983255
2)Норма невязки нулевого приближения
26.812442725370367
3)Оценка количества итераций
8.406677020422729
4)спектральный радиус
0.43968414403711953
5)
   iter  discrepancy   rel. d.     error  rel.error      diff  apost.est.  \
0     0    12.731538  0.474837  0.201849   0.207088  1.150800    0.903042   
1     1     5.187034  0.193456  0.115020   0.118005  0.290860    0.228240   
2     2     2.394266  0.089297  0.123390   0.126593  0.183938    0.144338   
3     3     1.022104  0.038121  0.115969   0.118980  0.064890    0.050919   
4     4     0.458722  0.017109  0.121217   0.124364  0.032907    0.025822   
5     5     0.199302  0.007433  0.119625   0.122730  0.013256    0.010402   
6     6     0.088386  0.003296  0.120462   0.123589  0.006167    0.004840   
7     7     0.038685  0.001443  0.120158   0.123277  0.002619    0.002055   
8     8     0